In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkb
import matplotlib.pyplot as plt
import folium
# === 1. Ler CSV ===

dados_altimetria_4326 = pd.read_csv('alt_curva_de_nivel.csv')
dados_linhas_4326 = pd.read_csv('dados_shapes_onibus_urbs_consolidado_threads.csv')
dados_pontos = pd.read_csv('dados_pontos_onibus_urbs_consolidado_threads.csv')



In [2]:
dados_pontos

,NOME,NUM,Latitude,Longitude,SEQ,GRUPO,SENTIDO,TIPO,ID_do_Itinerario,COD
0,"Rua Des. Westphalen, 1345 - Rebouças",110061,-25.444264,-49.267577,28,NaN,Guanabara,Novo mobiliário,13513,160
1,"Rua Lamenha Lins, 1592 - Rebouças",150721,-25.449942,-49.270820,32,NaN,Guanabara,Novo mobiliário,13513,160
2,"R. Myltho Anselmo da Silva, 1160 - Mercês",110134,-25.419373,-49.287072,15,NaN,Guanabara,Novo mobiliário,13513,160
3,"Rua Arthur Leinig, 785 - Vista Alegre",190977,-25.409838,-49.291969,9,NaN,Guanabara,Sem demarcação,13513,160
4,"Rua Alferes Poli, 787 - Rebouças",110024,-25.443478,-49.270141,13,NaN,Jd Mercês,Novo mobiliário,13533,160
...,...,...,...,...,...,...,...,...,...,...
13193,Av. Luiz Xavier - Boca Maldita,110206,-25.432103,-49.274256,29,NaN,Turismo,Linha Turismo,1065,979
13194,Memorial Árabe,110002,-25.423637,-49.268607,9,NaN,Turismo,Linha Turismo,1065,979
13195,"Rua Jacarezinho, 620 - Mercês",190211,-25.423812,-49.294582,26,NaN,Turismo,Linha Turismo,1065,979
13196,Bosque Alemão,190419,-25.407180,-49.284990,13,NaN,Turismo,Linha Turismo,10253,979


In [3]:
dados_linhas_4326 = dados_linhas_4326[dados_linhas_4326['COD'] == 500].reset_index(drop=True)
dados_linhas_4326 = dados_linhas_4326[dados_linhas_4326['SHP'] == 1940].reset_index(drop=True)

In [7]:
dados_linhas_4326 = dados_linhas_4326[dados_linhas_4326['COD'] == 521].reset_index(drop=True)
dados_linhas_4326 = dados_linhas_4326[dados_linhas_4326['SHP'] == 1964].reset_index(drop=True)

In [14]:
dados_linhas_4326

,SHP,Latitude,Longitude,COD
0,1940,-25.516848,-49.230057,500
1,1940,-25.516868,-49.230132,500
2,1940,-25.516920,-49.230100,500
3,1940,-25.517024,-49.230042,500
4,1940,-25.517186,-49.229950,500
...,...,...,...,...
555,1940,-25.433545,-49.269572,500
556,1940,-25.433583,-49.269682,500
557,1940,-25.433632,-49.269817,500
558,1940,-25.433678,-49.269951,500


In [4]:
def converte_srid(df):

  # Criar geometria de pontos a partir das colunas Longitude e Latitude
  gdf = gpd.GeoDataFrame(
      df,
      geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']),
      crs="EPSG:4326"  # Sistema original (WGS 84)
  )
  # Reprojetar para EPSG:3a1982 (SIRGAS 2000 / UTM zone 22S)
  gdf_utm = gdf.to_crs(epsg=31982)
  return gdf_utm


def define_crs_srid(df):
    from shapely import wkb
    import geopandas as gpd

    def safe_wkb_to_geom(hex_wkb):
        if isinstance(hex_wkb, str):
            return wkb.loads(bytes.fromhex(hex_wkb))
        return None

    df['geometry'] = df['geom'].apply(safe_wkb_to_geom)

    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.set_crs(epsg=31982, inplace=True)  # ⚠️ Define sem reprojetar
    return gdf


dados_linhas = converte_srid(dados_linhas_4326)
dados_altimetria = define_crs_srid(dados_altimetria_4326)
#dados_linhas_uni = dados_linhas[dados_linhas['SHP'] == 1940].reset_index(drop=True)
#dados_linhas_uni = dados_linhas_uni.head(50000).reset_index(drop=True)

In [16]:
from shapely.strtree import STRtree
from shapely.geometry.base import BaseGeometry

# 1. Lista de geometrias
geoms_linhas = list(dados_altimetria['geometry'])

# 2. Criar árvore
tree = STRtree(geoms_linhas)

# 3. Inicializar listas
idx_mais_proxima = []
idx_segunda_mais_proxima = []

# 4. Definir raio de busca (em metros)
raio_metros = 50  # Ajuste conforme seu CRS

for ponto in dados_linhas['geometry']:
    # Obter índices das curvas candidatas no buffer
    idx_candidatos = tree.query(ponto.buffer(raio_metros))  # Retorna índices (int) na versão <2.0

    # Obter as geometrias reais usando os índices
    geoms_candidatas = [geoms_linhas[i] for i in idx_candidatos if isinstance(geoms_linhas[i], BaseGeometry)]

    # Ordenar pelas distâncias
    ordenadas = sorted(zip(idx_candidatos, geoms_candidatas), key=lambda tup: ponto.distance(tup[1]))

    # Adicionar os dois índices mais próximos
    if len(ordenadas) >= 2:
        idx_mais_proxima.append(ordenadas[0][0])
        idx_segunda_mais_proxima.append(ordenadas[1][0])
    elif len(ordenadas) == 1:
        idx_mais_proxima.append(ordenadas[0][0])
        idx_segunda_mais_proxima.append(None)
    else:
        idx_mais_proxima.append(None)
        idx_segunda_mais_proxima.append(None)

# 5. Adicionar ao DataFrame
dados_linhas['idx_mais_proxima'] = idx_mais_proxima
dados_linhas['idx_segunda_mais_proxima'] = idx_segunda_mais_proxima

# 6. Opcional: extrair dados das curvas mais próximas
linhas1 = dados_altimetria.loc[dados_linhas['idx_mais_proxima'].dropna().astype(int)].reset_index(drop=True)
linhas2 = dados_altimetria.loc[dados_linhas['idx_segunda_mais_proxima'].dropna().astype(int)].reset_index(drop=True)

# 7. Concatenar
resultado = pd.concat([
    dados_linhas.reset_index(drop=True),
    linhas1.add_prefix('linha1_'),
    linhas2.add_prefix('linha2_')
], axis=1)


In [55]:
from shapely.strtree import STRtree
import numpy as np
import pandas as pd

# 1. Converter para lista de geometrias
geoms_linhas = dados_altimetria['geometry'].tolist()

# 2. Criar árvore
tree = STRtree(geoms_linhas)

# 3. Pré-alocar arrays para os índices
n_pontos = len(dados_linhas)
idx_mais_proxima = np.full(n_pontos, -1, dtype=int)
idx_segunda_mais_proxima = np.full(n_pontos, -1, dtype=int)

# 4. Definir raio de busca
raio_metros = 50

# 5. Iteração otimizada
for i, ponto in enumerate(dados_linhas['geometry']):
    # Encontrar geometrias dentro do buffer
    candidatos = tree.query(ponto.buffer(raio_metros))
    
    if len(candidatos) > 0:
        # Calcular distâncias
        distancias = [ponto.distance(geoms_linhas[idx]) for idx in candidatos]
        
        # Obter índices ordenados
        ordenados = [idx for _, idx in sorted(zip(distancias, candidatos))]
        
        # Armazenar resultados
        idx_mais_proxima[i] = ordenados[0]
        if len(ordenados) > 1:
            idx_segunda_mais_proxima[i] = ordenados[1]

# 6. Converter para Series com NaN onde não houve match
dados_linhas = dados_linhas.copy()
dados_linhas['idx_mais_proxima'] = [idx if idx != -1 else np.nan for idx in idx_mais_proxima]
dados_linhas['idx_segunda_mais_proxima'] = [idx if idx != -1 else np.nan for idx in idx_segunda_mais_proxima]

# 7. Extração segura de dados - MANTENDO A ORIGINAL QUE FUNCIONA
linhas1 = dados_altimetria.loc[dados_linhas['idx_mais_proxima'].dropna().astype(int)].reset_index(drop=True)
linhas2 = dados_altimetria.loc[dados_linhas['idx_segunda_mais_proxima'].dropna().astype(int)].reset_index(drop=True)

# 8. Concatenar como no código original
resultado = pd.concat([
    dados_linhas.reset_index(drop=True),
    linhas1.add_prefix('linha1_'),
    linhas2.add_prefix('linha2_')
], axis=1)

In [5]:
from shapely.strtree import STRtree
from shapely.geometry import Point, LineString
import numpy as np
import pandas as pd

# 1. Converter para lista de geometrias
geoms_linhas = dados_altimetria['geometry'].tolist()

# 2. Criar árvore
tree = STRtree(geoms_linhas)

# 3. Pré-alocar arrays para os índices
n_pontos = len(dados_linhas)
idx_mais_proxima = np.full(n_pontos, -1, dtype=int)
idx_segunda_mais_proxima = np.full(n_pontos, -1, dtype=int)

# 4. Definir raio de busca
raio_metros = 300

# 5. Iteração otimizada com seleção de curvas opostas
for i, ponto in enumerate(dados_linhas['geometry']):
    # Encontrar geometrias dentro do buffer
    candidatos = tree.query(ponto.buffer(raio_metros))
    
    if len(candidatos) > 0:
        # Calcular distâncias e pontos mais próximos
        distancias = []
        pontos_mais_proximos = []
        
        for idx in candidatos:
            linha = geoms_linhas[idx]
            ponto_prox = linha.interpolate(linha.project(ponto))
            dist = ponto.distance(ponto_prox)
            distancias.append(dist)
            pontos_mais_proximos.append(ponto_prox)
        
        # Ordenar candidatos por distância
        candidatos_ordenados = [idx for _, idx in sorted(zip(distancias, candidatos))]
        pontos_ordenados = [p for _, p in sorted(zip(distancias, pontos_mais_proximos))]
        
        # Selecionar a curva mais próxima
        idx_mais_proxima[i] = candidatos_ordenados[0]
        ponto_base = pontos_ordenados[0]
        
        # Encontrar curva oposta (do outro lado do ponto)
        for j in range(1, len(candidatos_ordenados)):
            idx_candidato = candidatos_ordenados[j]
            ponto_candidato = pontos_ordenados[j]
            
            # Calcular vetores para determinar o lado
            vetor_base = np.array([ponto.x - ponto_base.x, ponto.y - ponto_base.y])
            vetor_candidato = np.array([ponto_candidato.x - ponto_base.x, ponto_candidato.y - ponto_base.y])
            
            # Produto vetorial para determinar o lado (z-component)
            produto_vetorial = vetor_base[0] * vetor_candidato[1] - vetor_base[1] * vetor_candidato[0]
            
            # Se estiverem em lados opostos (produto vetorial com sinais diferentes)
            if abs(produto_vetorial) > 1e-8:  # Evitar erros numéricos
                idx_segunda_mais_proxima[i] = idx_candidato
                break
        else:
            # Se não encontrou curva oposta, usar a segunda mais próxima
            if len(candidatos_ordenados) > 1:
                idx_segunda_mais_proxima[i] = candidatos_ordenados[1]

# 6. Converter para Series com NaN onde não houve match
dados_linhas = dados_linhas.copy()
dados_linhas['idx_mais_proxima'] = [idx if idx != -1 else np.nan for idx in idx_mais_proxima]
dados_linhas['idx_segunda_mais_proxima'] = [idx if idx != -1 else np.nan for idx in idx_segunda_mais_proxima]

# 7. Extração de dados com tratamento de índices ausentes
linhas1 = pd.DataFrame()
linhas2 = pd.DataFrame()

# Preencher com dados reais onde existem índices válidos
if not dados_linhas['idx_mais_proxima'].isna().all():
    validos = dados_linhas['idx_mais_proxima'].dropna().astype(int)
    linhas1 = dados_altimetria.loc[validos].reset_index(drop=True)
    linhas1 = linhas1.add_prefix('linha1_')

if not dados_linhas['idx_segunda_mais_proxima'].isna().all():
    validos = dados_linhas['idx_segunda_mais_proxima'].dropna().astype(int)
    linhas2 = dados_altimetria.loc[validos].reset_index(drop=True)
    linhas2 = linhas2.add_prefix('linha2_')

# 8. Concatenar resultados garantindo alinhamento
resultado = pd.concat([
    dados_linhas.reset_index(drop=True),
    linhas1,
    linhas2
], axis=1)

# Preencher NaN para colunas faltantes
for col in linhas1.columns:
    if col not in resultado:
        resultado[col] = np.nan
for col in linhas2.columns:
    if col not in resultado:
        resultado[col] = np.nan

In [6]:
resultado

,SHP,Latitude,Longitude,COD,geometry,idx_mais_proxima,idx_segunda_mais_proxima,linha1_gid,linha1_layer,linha1_elevation,...,linha2_layer,linha2_elevation,linha2_data_voo,linha2_data_rest,linha2_resp_tec,linha2_escala,linha2_curvas_m,linha2_shape_len,linha2_geom,linha2_geometry
0,1940,-25.516848,-49.230057,500,POINT (677867.964 7176636.679),19598,15454,115013,CURVA INTERMEDIARIA,881.0,...,CURVA INTERMEDIARIA,881.0,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,68.800874,0105000020E61000000100000001020000000D00000000...,"MULTILINESTRING ((677868.663 7176571.723, 6778..."
1,1940,-25.516868,-49.230132,500,POINT (677860.394 7176634.601),19598,19674,115013,CURVA INTERMEDIARIA,881.0,...,CURVA INTERMEDIARIA,882.0,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,2914.147072,0105000020E61000000100000001020000003601000000...,"MULTILINESTRING ((677975.053 7177199.693, 6779..."
2,1940,-25.516920,-49.230100,500,POINT (677863.492 7176628.828),19598,15454,115013,CURVA INTERMEDIARIA,881.0,...,CURVA INTERMEDIARIA,881.0,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,68.800874,0105000020E61000000100000001020000000D00000000...,"MULTILINESTRING ((677868.663 7176571.723, 6778..."
3,1940,-25.517024,-49.230042,500,POINT (677869.217 7176617.221),19598,15454,115013,CURVA INTERMEDIARIA,881.0,...,CURVA INTERMEDIARIA,881.0,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,68.800874,0105000020E61000000100000001020000000D00000000...,"MULTILINESTRING ((677868.663 7176571.723, 6778..."
4,1940,-25.517186,-49.229950,500,POINT (677878.173 7176599.136),15454,19598,115086,CURVA INTERMEDIARIA,881.0,...,CURVA INTERMEDIARIA,881.0,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,2695.441565,0105000020E61000000100000001020000004F01000000...,"MULTILINESTRING ((678002.153 7177108.563, 6780..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,1940,-25.433545,-49.269572,500,POINT (674016.047 7185916.227),21013,22035,116421,CURVA MESTRA,895.0,...,CURVA INTERMEDIARIA,894.0,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,2941.633485,0105000020E6100000010000000102000000F801000000...,"MULTILINESTRING ((674443.723 7186799.693, 6744..."
556,1940,-25.433583,-49.269682,500,POINT (674004.99 7185912.217),21013,22035,116421,CURVA MESTRA,895.0,...,CURVA INTERMEDIARIA,894.0,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,2941.633485,0105000020E6100000010000000102000000F801000000...,"MULTILINESTRING ((674443.723 7186799.693, 6744..."
557,1940,-25.433632,-49.269817,500,POINT (673991.285 7185906.934),21013,22035,116421,CURVA MESTRA,895.0,...,CURVA INTERMEDIARIA,894.0,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,2941.633485,0105000020E6100000010000000102000000F801000000...,"MULTILINESTRING ((674443.723 7186799.693, 6744..."
558,1940,-25.433678,-49.269951,500,POINT (673977.747 7185901.971),21013,22035,116421,CURVA MESTRA,895.0,...,CURVA INTERMEDIARIA,894.0,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,2941.633485,0105000020E6100000010000000102000000F801000000...,"MULTILINESTRING ((674443.723 7186799.693, 6744..."


In [22]:
resultado1 = resultado[resultado['COD'] == 22]

In [20]:
resultado1

NameError: name 'resultado1' is not defined

In [22]:
#resultado2 = resultado1.head(10)
resultado1 = resultado[resultado['SHP'] == 1940]

In [8]:
# Função para calcular a elevação interpolada para uma única linha
import numpy as np
def interpolar_altimetria(row):
    ponto = row['geometry']
    linha1_geom = row['linha1_geometry']
    linha2_geom = row['linha2_geometry']
    elevacao1 = row['linha1_elevation']
    elevacao2 = row['linha2_elevation']

    # Lidar com casos onde as geometrias ou elevações são nulas
    if pd.isna(elevacao1) and pd.isna(elevacao2):
        return np.nan # Não há dados para interpolar

    if pd.isna(elevacao1) and not pd.isna(elevacao2):
        return elevacao2 # Se apenas uma elevação estiver disponível, use-a
    if not pd.isna(elevacao1) and pd.isna(elevacao2):
        return elevacao1 # Se apenas uma elevação estiver disponível, use-a

    # Se ambas as elevações são válidas, prossiga com a interpolação ponderada
    if linha1_geom and linha2_geom:
        distancia1 = ponto.distance(linha1_geom)
        distancia2 = ponto.distance(linha2_geom)

        # Evitar divisão por zero se ambas as distâncias forem zero (ponto está sobre ambas as linhas)
        if distancia1 == 0 and distancia2 == 0:
            return (elevacao1 + elevacao2) / 2
        elif distancia1 == 0:
            return elevacao1 # Ponto está na linha 1
        elif distancia2 == 0:
            return elevacao2 # Ponto está na linha 2

        # Fórmula de interpolação ponderada inversa da distância (IDW)
        # Mais próximo significa maior peso.
        # Peso da linha 1 = 1/distancia1
        # Peso da linha 2 = 1/distancia2
        peso1 = 1 / distancia1
        peso2 = 1 / distancia2

        elevacao_interpolada = (elevacao1 * peso1 + elevacao2 * peso2) / (peso1 + peso2)
        return elevacao_interpolada
    else:
        return np.nan # Geometrias nulas ou inválidas

# Aplicar a função a cada linha do DataFrame
# Usamos .copy() para evitar SettingWithCopyWarning
resultado_copia = resultado.copy()
resultado_copia['altimetria_interpolada'] = resultado_copia.apply(interpolar_altimetria, axis=1)

In [9]:
#resultado_copia = resultado_copia.head(200)


# 1. Calcular o centro do mapa
lat_center = resultado_copia['Latitude'].mean()
lon_center = resultado_copia['Longitude'].mean()

# 2. Criar mapa Folium centrado
m = folium.Map(location=[lat_center, lon_center], zoom_start=15)

# 3. Adicionar pontos no mapa
for idx, row in resultado_copia.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3,
        color='blue',
        fill=True,
        fill_opacity=0.7,
        popup=f"SHP: {row['SHP']}, COD: {row['COD']}"
    ).add_to(m)

# 4. Exibir mapa
m


In [14]:

# Calcular as diferenças absolutas da coluna elevation entre pontos consecutivos
resultado_copia['diff_elev'] = resultado_copia['altimetria_interpolada'].diff().abs()

# Somar todas as diferenças para obter a variação total altimétrica
variacao_total = resultado_copia['diff_elev'].sum()

print("Variação para linha 500 em metros: ", variacao_total)

Variação para linha 500 em metros:  176.6362814867581


In [54]:
resultado_copia = resultado_copia.reset_index(drop=True)

In [26]:
resultado1 = resultado1.reset_index(drop=True)

NameError: name 'resultado1' is not defined

In [30]:
#df = resultado1.iloc[[2]].copy()
#df = df.reset_index(drop=True)

In [27]:
resultado_copia

,SHP,Latitude,Longitude,COD,geometry,idx_mais_proxima,idx_segunda_mais_proxima,linha1_gid,linha1_layer,linha1_elevation,...,linha2_data_voo,linha2_data_rest,linha2_resp_tec,linha2_escala,linha2_curvas_m,linha2_shape_len,linha2_geom,linha2_geometry,altimetria_interpolada,diff_elev
0,4146,-25.406631,-49.252164,22,POINT (675806.082 7188874.605),27337,27342,122759,CURVA INTERMEDIARIA,912.0,...,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,1656.956598,0105000020E61000000100000001020000000401000000...,"MULTILINESTRING ((675637.999 7188999.693, 6756...",912.315568,NaN
1,4146,-25.406486,-49.252032,22,POINT (675819.601 7188890.491),27337,26830,122759,CURVA INTERMEDIARIA,912.0,...,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,1639.730080,0105000020E61000000100000001020000000801000000...,"MULTILINESTRING ((675841.483 7188999.693, 6758...",911.791885,0.523682
2,4146,-25.406337,-49.251887,22,POINT (675834.373 7188906.805),26830,26361,122251,CURVA INTERMEDIARIA,911.0,...,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,1056.215358,0105000020E6100000010000000102000000E700000000...,"MULTILINESTRING ((675977.209 7188999.693, 6759...",910.773833,1.018052
3,4146,-25.406844,-49.251460,22,POINT (675876.586 7188850.091),26361,26830,121787,CURVA MESTRA,910.0,...,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,1639.730080,0105000020E61000000100000001020000000801000000...,"MULTILINESTRING ((675841.483 7188999.693, 6758...",910.464714,0.309119
4,4146,-25.407485,-49.250905,22,POINT (675931.528 7188778.41),26361,26830,121787,CURVA MESTRA,910.0,...,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,1639.730080,0105000020E61000000100000001020000000801000000...,"MULTILINESTRING ((675841.483 7188999.693, 6758...",910.426158,0.038557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,4146,-25.407225,-49.252731,22,POINT (675748.165 7188809.598),27012,25033,122435,CURVA MESTRA,915.0,...,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,1813.690400,0105000020E61000000100000001020000000D01000000...,"MULTILINESTRING ((675612.237 7188999.693, 6756...",914.736353,1.955901
969,4146,-25.407053,-49.252566,22,POINT (675764.983 7188828.429),25033,27012,120457,CURVA INTERMEDIARIA,914.0,...,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,718.615264,0105000020E6100000010000000102000000AB00000000...,"MULTILINESTRING ((675720.773 7188890.643, 6757...",914.097941,0.638412
970,4146,-25.406902,-49.252426,22,POINT (675779.355 7188844.917),27342,25033,122761,CURVA INTERMEDIARIA,913.0,...,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,1813.690400,0105000020E61000000100000001020000000D01000000...,"MULTILINESTRING ((675612.237 7188999.693, 6756...",913.459392,0.638549
971,4146,-25.406760,-49.252291,22,POINT (675793.141 7188860.539),27342,27337,122761,CURVA INTERMEDIARIA,913.0,...,1972 / NOV-1979,1972 / 1980,GEOFOTO SA / SAESP LTDA,1:2.000,1.0,1470.946414,0105000020E6100000010000000102000000EB00000000...,"MULTILINESTRING ((675768.674 7188999.693, 6757...",912.993179,0.466213


In [32]:
import folium
from folium import FeatureGroup, LayerControl
from shapely import wkt
import pandas as pd
from pyproj import Transformer, CRS # Importar pyproj

# ==============================================================================
# ATENÇÃO: Defina aqui o código EPSG do sistema de coordenadas dos seus dados de LINHA
# Exemplos comuns para a região de Curitiba (UTM Zone 22S):
# SOURCE_EPSG = "EPSG:31982"  # Para SIRGAS 2000 / UTM zone 22S
# SOURCE_EPSG = "EPSG:29192"  # Para SAD69 / UTM zone 22S
# Consulte a origem dos seus dados para saber o EPSG correto.
SOURCE_EPSG = "EPSG:31982" # <- MUDE AQUI SE NECESSÁRIO
TARGET_EPSG = "EPSG:4326"  # WGS84 (Latitude/Longitude) que o Folium usa
# ==============================================================================

# Criar o objeto Transformer uma vez para reutilização
try:
    transformer = Transformer.from_crs(CRS.from_string(SOURCE_EPSG), CRS.from_string(TARGET_EPSG), always_xy=True)
except Exception as e:
    print(f"Erro ao criar o transformador de coordenadas com pyproj: {e}")
    print(f"Verifique se o código SOURCE_EPSG ('{SOURCE_EPSG}') está correto e se pyproj está instalado.")
    exit()
'''
# Lendo o CSV
file_path = 'resultado1.csv'
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Erro: O arquivo '{file_path}' não foi encontrado.")
    exit()
'''
df = resultado_copia.copy()
if df.empty:
    print(f"Erro: O arquivo CSV '{file_path}' está vazio.")
    exit()

required_columns = ['Latitude', 'Longitude', 'linha1_geometry', 'linha2_geometry']
for col in required_columns:
    if col not in df.columns:
        print(f"Erro: A coluna '{col}' não foi encontrada no CSV. Colunas encontradas: {df.columns.tolist()}")
        exit()

# Obtendo coordenadas do ponto CENTRAL do mapa
# Certifique-se que estas coordenadas no CSV estão em graus decimais (Lat/Lon)
map_center_latitude = df.loc[0, 'Latitude']
map_center_longitude = df.loc[0, 'Longitude']
print(f"Centro do mapa definido para Latitude: {map_center_latitude}, Longitude: {map_center_longitude}")


# Criando o mapa centralizado no ponto
m = folium.Map(location=[map_center_latitude, map_center_longitude], zoom_start=15)

# ====== Camada do ponto ======
fg_ponto = FeatureGroup(name='Ponto Principal')
folium.Marker(
    location=[map_center_latitude, map_center_longitude],
    popup='Ponto Principal',
    icon=folium.Icon(color='red')
).add_to(fg_ponto)
fg_ponto.add_to(m)

# Função auxiliar para adicionar geometrias de linha com REPROJEÇÃO
def adicionar_geometria_linha(wkt_string, feature_group, cor_linha, nome_camada_debug):
    print(f"\nProcessando {nome_camada_debug}:")
    if pd.isna(wkt_string) or not str(wkt_string).strip():
        print(f"  Aviso: String WKT para '{nome_camada_debug}' está vazia ou é NaN. Camada pulada.")
        return

    wkt_string_preview = str(wkt_string)[:100] + ('...' if len(str(wkt_string)) > 100 else '')
    print(f"  WKT recebido: '{wkt_string_preview}'")

    try:
        geom = wkt.loads(str(wkt_string))
    except Exception as e:
        print(f"  Erro ao parsear WKT para '{nome_camada_debug}': {wkt_string_preview}. Erro: {e}")
        return

    if geom.is_empty:
        print(f"  Aviso: Geometria WKT parseada para '{nome_camada_debug}' está vazia. Camada pulada.")
        return

    print(f"  Tipo de geometria parseada: {geom.geom_type}")

    linhas_para_plotar_orig = []
    if geom.geom_type == 'LineString':
        linhas_para_plotar_orig.append(geom)
    elif geom.geom_type == 'MultiLineString':
        linhas_para_plotar_orig.extend(list(geom.geoms))
    else:
        print(f"  Aviso: Tipo de geometria não suportado '{geom.geom_type}' para '{nome_camada_debug}'. Camada pulada.")
        return

    if not linhas_para_plotar_orig:
        print(f"  Aviso: Nenhuma linha individual encontrada em '{nome_camada_debug}'.")
        return
        
    segmentos_adicionados = 0
    for i, linha_orig in enumerate(linhas_para_plotar_orig):
        if linha_orig.is_empty:
            print(f"  Aviso: Segmento {i} (original) de '{nome_camada_debug}' está vazio.")
            continue
        
        # Coordenadas em WKT projetado são (x, y) ou (Leste, Norte)
        coords_originais_projetadas = list(linha_orig.coords) # Lista de tuplas (x_proj, y_proj)
        
        if not coords_originais_projetadas:
            print(f"  Aviso: Segmento {i} (original) de '{nome_camada_debug}' não possui coordenadas.")
            continue
        
        # Reprojetar coordenadas
        coords_reprojetadas_folium = []
        try:
            for x_proj, y_proj in coords_originais_projetadas:
                # transformer.transform espera (x, y) e retorna (lon, lat) devido a always_xy=True
                lon_deg, lat_deg = transformer.transform(x_proj, y_proj)
                coords_reprojetadas_folium.append((lat_deg, lon_deg)) # Folium espera (lat, lon)
        except Exception as e:
            print(f"    Erro durante a reprojeção do segmento {i} de '{nome_camada_debug}': {e}")
            print(f"    Coordenadas problemáticas (primeiras 2): {coords_originais_projetadas[:2]}")
            continue # Pula para o próximo segmento se este falhar

        if not coords_reprojetadas_folium:
            print(f"  Aviso: Segmento {i} de '{nome_camada_debug}' não possui coordenadas após reprojeção.")
            continue
            
        # print(f"  Coordenadas reprojetadas para segmento {i} de '{nome_camada_debug}' (primeiras 2): {coords_reprojetadas_folium[:2]}...")
        folium.PolyLine(coords_reprojetadas_folium, color=cor_linha, weight=2.5, opacity=0.8).add_to(feature_group)
        segmentos_adicionados +=1
    
    if segmentos_adicionados > 0:
        print(f"  {segmentos_adicionados} segmento(s) de linha (reprojetados) adicionado(s) para '{nome_camada_debug}'.")
        if len(coords_reprojetadas_folium) > 0:
             print(f"    Exemplo de primeiras coordenadas reprojetadas (Lat, Lon): {coords_reprojetadas_folium[0]}")
    else:
        print(f"  Nenhum segmento de linha válido foi adicionado para '{nome_camada_debug}'.")


# ====== Camada Curva de Nível 1 ======
fg_curva1 = FeatureGroup(name='Curva de Nível 1', show=True)
wkt_linha1 = df.loc[0, 'linha1_geometry']
adicionar_geometria_linha(wkt_linha1, fg_curva1, 'blue', 'Curva de Nível 1')
fg_curva1.add_to(m)

# ====== Camada Curva de Nível 2 ======
fg_curva2 = FeatureGroup(name='Curva de Nível 2', show=True)
wkt_linha2 = df.loc[0, 'linha2_geometry']
adicionar_geometria_linha(wkt_linha2, fg_curva2, 'green', 'Curva de Nível 2')
fg_curva2.add_to(m)

# ====== Controle de Camadas ======
LayerControl().add_to(m)

output_map_file = 'mapa_curvas_revisado_reprojetado.html'
m.save(output_map_file)
print(f"\nMapa salvo como '{output_map_file}'. Abra este arquivo em um navegador para visualizar.")
# m # Descomente para exibir no Jupyter Notebook

Centro do mapa definido para Latitude: -25.516848128860715, Longitude: -49.23005693809807

Processando Curva de Nível 1:
  WKT recebido: 'MULTILINESTRING ((675208.1430053711 7182001.752990723, 675207.1430053711 7182005.733032227, 675205.9...'
  Tipo de geometria parseada: MultiLineString
  1 segmento(s) de linha (reprojetados) adicionado(s) para 'Curva de Nível 1'.
    Exemplo de primeiras coordenadas reprojetadas (Lat, Lon): (-25.468738598029258, -49.25721360195566)

Processando Curva de Nível 2:
  WKT recebido: 'MULTILINESTRING ((674142.0330200195 7186799.692993164, 674144.5029907227 7186796.023010254, 674146.2...'
  Tipo de geometria parseada: MultiLineString
  1 segmento(s) de linha (reprojetados) adicionado(s) para 'Curva de Nível 2'.
    Exemplo de primeiras coordenadas reprojetadas (Lat, Lon): (-25.425555988155054, -49.268433991209015)

Mapa salvo como 'mapa_curvas_revisado_reprojetado.html'. Abra este arquivo em um navegador para visualizar.


In [33]:
import folium
from folium import FeatureGroup, LayerControl
from shapely import wkt
import pandas as pd
from pyproj import Transformer, CRS # Importar pyproj

# ==============================================================================
# ATENÇÃO: Defina aqui o código EPSG do sistema de coordenadas dos seus dados de LINHA
# Exemplos comuns para a região de Curitiba (UTM Zone 22S):
# SOURCE_EPSG = "EPSG:31982"  # Para SIRGAS 2000 / UTM zone 22S
# SOURCE_EPSG = "EPSG:29192"  # Para SAD69 / UTM zone 22S
# Consulte a origem dos seus dados para saber o EPSG correto.
SOURCE_EPSG = "EPSG:31982" # <- MUDE AQUI SE NECESSÁRIO
TARGET_EPSG = "EPSG:4326"  # WGS84 (Latitude/Longitude) que o Folium usa
# ==============================================================================

df = resultado_copia.copy()

if df.empty:
    print(f"Erro: O DataFrame carregado de '{file_path}' está vazio.")
    exit()

# --- Verificação das colunas obrigatórias ---
required_columns = ['Latitude', 'Longitude', 'linha1_geometry', 'linha2_geometry']
for col in required_columns:
    if col not in df.columns:
        print(f"Erro: A coluna '{col}' não foi encontrada no CSV. Colunas encontradas: {df.columns.tolist()}")
        exit()

# --- Definição do centro do mapa (usando a primeira linha) ---
# Você pode alterar esta lógica se precisar de um centro diferente (ex: média das coordenadas)
map_center_latitude = df.loc[0, 'Latitude']
map_center_longitude = df.loc[0, 'Longitude']
print(f"Centro do mapa definido para Latitude: {map_center_latitude}, Longitude: {map_center_longitude} (da primeira linha do CSV)")

# --- Criação do mapa ---
# Ajustei o zoom_start para 12, pode ser necessário ajustar dependendo da dispersão dos seus dados
m = folium.Map(location=[map_center_latitude, map_center_longitude], zoom_start=12)

# --- Criação dos FeatureGroups gerais ---
# Um grupo para todos os marcadores de pontos
fg_pontos_todos = FeatureGroup(name='Todos os Pontos Principais', show=True)
m.add_child(fg_pontos_todos)

# Um grupo para todas as geometrias de "linha1"
fg_curva1_todas = FeatureGroup(name='Todas Curvas de Nível 1', show=True)
m.add_child(fg_curva1_todas)

# Um grupo para todas as geometrias de "linha2"
fg_curva2_todas = FeatureGroup(name='Todas Curvas de Nível 2', show=True)
m.add_child(fg_curva2_todas)


# Função auxiliar para adicionar geometrias de linha com REPROJEÇÃO (sem alterações nesta função)
def adicionar_geometria_linha(wkt_string, feature_group, cor_linha, nome_camada_debug):
    print(f"\nProcessando {nome_camada_debug}:")
    if pd.isna(wkt_string) or not str(wkt_string).strip():
        print(f"  Aviso: String WKT para '{nome_camada_debug}' está vazia ou é NaN. Camada pulada.")
        return

    wkt_string_preview = str(wkt_string)[:100] + ('...' if len(str(wkt_string)) > 100 else '')
    print(f"  WKT recebido: '{wkt_string_preview}'")

    try:
        geom = wkt.loads(str(wkt_string))
    except Exception as e:
        print(f"  Erro ao parsear WKT para '{nome_camada_debug}': {wkt_string_preview}. Erro: {e}")
        return

    if geom.is_empty:
        print(f"  Aviso: Geometria WKT parseada para '{nome_camada_debug}' está vazia. Camada pulada.")
        return

    print(f"  Tipo de geometria parseada: {geom.geom_type}")

    linhas_para_plotar_orig = []
    if geom.geom_type == 'LineString':
        linhas_para_plotar_orig.append(geom)
    elif geom.geom_type == 'MultiLineString':
        linhas_para_plotar_orig.extend(list(geom.geoms))
    else:
        print(f"  Aviso: Tipo de geometria não suportado '{geom.geom_type}' para '{nome_camada_debug}'. Camada pulada.")
        return

    if not linhas_para_plotar_orig:
        print(f"  Aviso: Nenhuma linha individual encontrada em '{nome_camada_debug}'.")
        return
        
    segmentos_adicionados = 0
    coords_reprojetadas_folium_exemplo = [] # Para imprimir exemplo
    for i, linha_orig in enumerate(linhas_para_plotar_orig):
        if linha_orig.is_empty:
            print(f"  Aviso: Segmento {i} (original) de '{nome_camada_debug}' está vazio.")
            continue
        
        coords_originais_projetadas = list(linha_orig.coords)
        
        if not coords_originais_projetadas:
            print(f"  Aviso: Segmento {i} (original) de '{nome_camada_debug}' não possui coordenadas.")
            continue
        
        coords_reprojetadas_folium = []
        try:
            for x_proj, y_proj in coords_originais_projetadas:
                lon_deg, lat_deg = transformer.transform(x_proj, y_proj)
                coords_reprojetadas_folium.append((lat_deg, lon_deg))
        except Exception as e:
            print(f"    Erro durante a reprojeção do segmento {i} de '{nome_camada_debug}': {e}")
            print(f"    Coordenadas problemáticas (primeiras 2): {coords_originais_projetadas[:2]}")
            continue

        if not coords_reprojetadas_folium:
            print(f"  Aviso: Segmento {i} de '{nome_camada_debug}' não possui coordenadas após reprojeção.")
            continue
        
        if not coords_reprojetadas_folium_exemplo: # Salva o primeiro conjunto de coords para exemplo
            coords_reprojetadas_folium_exemplo = coords_reprojetadas_folium

        folium.PolyLine(coords_reprojetadas_folium, color=cor_linha, weight=2.5, opacity=0.8).add_to(feature_group)
        segmentos_adicionados +=1
    
    if segmentos_adicionados > 0:
        print(f"  {segmentos_adicionados} segmento(s) de linha (reprojetados) adicionado(s) para '{nome_camada_debug}'.")
        if coords_reprojetadas_folium_exemplo and coords_reprojetadas_folium_exemplo[0]:
             print(f"    Exemplo de primeiras coordenadas reprojetadas (Lat, Lon): {coords_reprojetadas_folium_exemplo[0]}")
    else:
        print(f"  Nenhum segmento de linha válido foi adicionado para '{nome_camada_debug}'.")

# --- Iterar sobre cada linha (tupla) do DataFrame ---
for index, row in df.iterrows():
    print(f"\n--- Processando dados da Linha {index} do CSV ---")
    
    current_lat = row['Latitude']
    current_lon = row['Longitude']
    
    # Adicionar marcador para o ponto principal da linha atual
    popup_text = f"Ponto ID {index}<br>Lat: {current_lat:.5f}<br>Lon: {current_lon:.5f}"
    folium.Marker(
        location=[current_lat, current_lon],
        popup=popup_text,
        icon=folium.Icon(color='purple', icon='info-sign') # Cor diferente para distinguir dos antigos
    ).add_to(fg_pontos_todos)

    # Processar e adicionar Curva de Nível 1 para a linha atual
    wkt_l1 = row['linha1_geometry']
    debug_name_l1 = f"Curva Nível 1 (CSV Linha {index})"
    adicionar_geometria_linha(wkt_l1, fg_curva1_todas, 'blue', debug_name_l1)

    # Processar e adicionar Curva de Nível 2 para a linha atual
    wkt_l2 = row['linha2_geometry']
    debug_name_l2 = f"Curva Nível 2 (CSV Linha {index})"
    adicionar_geometria_linha(wkt_l2, fg_curva2_todas, 'green', debug_name_l2)


# ====== Controle de Camadas ======
LayerControl().add_to(m)

# --- Salvamento do mapa ---
output_map_file = 'mapa_curvas_multiplas_linhas.html'
m.save(output_map_file)
print(f"\nMapa salvo como '{output_map_file}'. Abra este arquivo em um navegador para visualizar.")
# m # Descomente para exibir no Jupyter Notebook

Centro do mapa definido para Latitude: -25.516848128860715, Longitude: -49.23005693809807 (da primeira linha do CSV)

--- Processando dados da Linha 0 do CSV ---

Processando Curva Nível 1 (CSV Linha 0):
  WKT recebido: 'MULTILINESTRING ((675208.1430053711 7182001.752990723, 675207.1430053711 7182005.733032227, 675205.9...'
  Tipo de geometria parseada: MultiLineString
  1 segmento(s) de linha (reprojetados) adicionado(s) para 'Curva Nível 1 (CSV Linha 0)'.
    Exemplo de primeiras coordenadas reprojetadas (Lat, Lon): (-25.468738598029258, -49.25721360195566)

Processando Curva Nível 2 (CSV Linha 0):
  WKT recebido: 'MULTILINESTRING ((674142.0330200195 7186799.692993164, 674144.5029907227 7186796.023010254, 674146.2...'
  Tipo de geometria parseada: MultiLineString
  1 segmento(s) de linha (reprojetados) adicionado(s) para 'Curva Nível 2 (CSV Linha 0)'.
    Exemplo de primeiras coordenadas reprojetadas (Lat, Lon): (-25.425555988155054, -49.268433991209015)

--- Processando dados da Lin

In [34]:
import folium
from folium import FeatureGroup, LayerControl
from shapely import wkt
import pandas as pd
from pyproj import Transformer, CRS

# ==============================================================================
# CONFIGURAÇÕES DO USUÁRIO - AJUSTE CONFORME NECESSÁRIO
# ==============================================================================

# 1. Códigos EPSG para reprojeção de coordenadas das LINHAS
# Consulte a origem dos seus dados para saber o EPSG correto das geometrias de linha.
SOURCE_EPSG = "EPSG:31982"  # Ex: SIRGAS 2000 / UTM zone 22S (ajuste se necessário)
TARGET_EPSG = "EPSG:4326"    # WGS84 (Latitude/Longitude) que o Folium usa

# 2. Nomes das colunas de altitude no seu arquivo CSV
# Estes devem corresponder aos nomes de colunas identificados no seu arquivo:
ALTITUDE_PONTO_COL = 'altimetria_interpolada' # Altitude do ponto principal
ALTITUDE_LINHA1_COL = 'linha1_elevation'     # Altitude da primeira curva de nível associada
ALTITUDE_LINHA2_COL = 'linha2_elevation'     # Altitude da segunda curva de nível associada
UNIDADE_ALTITUDE = "m" # Ex: "metros", "unidades" - usado nos popups

# 3. Caminho para o arquivo CSV
FILE_PATH_CSV = 'resultado1.csv'

# 4. Nome do arquivo HTML de saída do mapa
OUTPUT_MAP_FILE = 'mapa_pontos_com_associacao_curvas.html'
# ==============================================================================

# Criar o objeto Transformer
try:
    transformer = Transformer.from_crs(CRS.from_string(SOURCE_EPSG), CRS.from_string(TARGET_EPSG), always_xy=True)
except Exception as e:
    print(f"Erro ao criar o transformador de coordenadas com pyproj: {e}")
    print(f"Verifique se o código SOURCE_EPSG ('{SOURCE_EPSG}') está correto e se pyproj está instalado.")
    exit()

df = resultado_copia.copy()

if df.empty:
    print(f"Erro: O DataFrame carregado de '{FILE_PATH_CSV}' está vazio.")
    exit()

# Verificação das colunas obrigatórias (incluindo altitudes)
required_columns = ['Latitude', 'Longitude', 'linha1_geometry', 'linha2_geometry', # Geometrias
                    ALTITUDE_PONTO_COL, ALTITUDE_LINHA1_COL, ALTITUDE_LINHA2_COL]   # Altitudes
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    print(f"Erro: As seguintes colunas obrigatórias não foram encontradas no CSV: {', '.join(missing_cols)}")
    print(f"Colunas encontradas no CSV: {df.columns.tolist()}")
    print(f"Verifique os nomes das colunas definidos em ALTITUDE_PONTO_COL, ALTITUDE_LINHA1_COL, ALTITUDE_LINHA2_COL.")
    exit()

# Definição do centro do mapa (usando a primeira linha)
map_center_latitude = df.loc[0, 'Latitude']
map_center_longitude = df.loc[0, 'Longitude']
print(f"Centro do mapa: Lat: {map_center_latitude}, Lon: {map_center_longitude} (da primeira linha do CSV)")

# Criação do mapa
m = folium.Map(location=[map_center_latitude, map_center_longitude], zoom_start=12)

# FeatureGroups gerais
fg_pontos_todos = FeatureGroup(name='Pontos Principais', show=True)
m.add_child(fg_pontos_todos)
fg_curva1_todas = FeatureGroup(name='Curvas de Nível 1', show=True)
m.add_child(fg_curva1_todas)
fg_curva2_todas = FeatureGroup(name='Curvas de Nível 2', show=True)
m.add_child(fg_curva2_todas)

# Função auxiliar para adicionar geometrias de linha com REPROJEÇÃO e ALTITUDE
def adicionar_geometria_linha(wkt_string, feature_group, cor_linha, nome_camada_debug, altitude_valor):
    print(f"\nProcessando {nome_camada_debug}:")
    if pd.isna(wkt_string) or not str(wkt_string).strip():
        print(f"  Aviso: String WKT para '{nome_camada_debug}' está vazia ou é NaN. Camada pulada.")
        return

    altitude_display = f"{altitude_valor} {UNIDADE_ALTITUDE}" if pd.notna(altitude_valor) else "N/A"
    popup_text_linha = f"Altitude da Curva: {altitude_display}" # Popup para a própria linha
    tooltip_text_linha = f"Alt: {altitude_display}" # Tooltip para a própria linha

    wkt_string_preview = str(wkt_string)[:100] + ('...' if len(str(wkt_string)) > 100 else '')
    print(f"  WKT recebido: '{wkt_string_preview}', Altitude associada à linha: {altitude_display}")

    try:
        geom = wkt.loads(str(wkt_string))
    except Exception as e:
        print(f"  Erro ao parsear WKT para '{nome_camada_debug}': {wkt_string_preview}. Erro: {e}")
        return

    if geom.is_empty:
        print(f"  Aviso: Geometria WKT parseada para '{nome_camada_debug}' está vazia. Camada pulada.")
        return
    print(f"  Tipo de geometria parseada: {geom.geom_type}")

    linhas_para_plotar_orig = []
    if geom.geom_type == 'LineString':
        linhas_para_plotar_orig.append(geom)
    elif geom.geom_type == 'MultiLineString':
        linhas_para_plotar_orig.extend(list(geom.geoms))
    else:
        print(f"  Aviso: Tipo de geometria não suportado '{geom.geom_type}' para '{nome_camada_debug}'. Camada pulada.")
        return

    if not linhas_para_plotar_orig:
        print(f"  Aviso: Nenhuma linha individual encontrada em '{nome_camada_debug}'.")
        return
        
    segmentos_adicionados = 0
    coords_reprojetadas_folium_exemplo = []
    for i, linha_orig in enumerate(linhas_para_plotar_orig):
        if linha_orig.is_empty:
            print(f"  Aviso: Segmento {i} (original) de '{nome_camada_debug}' está vazio.")
            continue
        
        coords_originais_projetadas = list(linha_orig.coords)
        if not coords_originais_projetadas:
            print(f"  Aviso: Segmento {i} (original) de '{nome_camada_debug}' não possui coordenadas.")
            continue
        
        coords_reprojetadas_folium = []
        try:
            for x_proj, y_proj in coords_originais_projetadas:
                lon_deg, lat_deg = transformer.transform(x_proj, y_proj)
                coords_reprojetadas_folium.append((lat_deg, lon_deg))
        except Exception as e:
            print(f"    Erro durante reprojeção do segmento {i} de '{nome_camada_debug}': {e}")
            continue

        if not coords_reprojetadas_folium:
            print(f"  Aviso: Segmento {i} de '{nome_camada_debug}' não possui coordenadas após reprojeção.")
            continue
        
        if not coords_reprojetadas_folium_exemplo:
            coords_reprojetadas_folium_exemplo = coords_reprojetadas_folium

        folium.PolyLine(
            coords_reprojetadas_folium,
            color=cor_linha,
            weight=2.5,
            opacity=0.8,
            popup=popup_text_linha, 
            tooltip=tooltip_text_linha 
        ).add_to(feature_group)
        segmentos_adicionados +=1
    
    if segmentos_adicionados > 0:
        print(f"  {segmentos_adicionados} segmento(s) de linha adicionado(s) para '{nome_camada_debug}'.")
        if coords_reprojetadas_folium_exemplo and coords_reprojetadas_folium_exemplo[0]:
             print(f"    Exemplo de primeiras coordenadas reprojetadas (Lat, Lon): {coords_reprojetadas_folium_exemplo[0]}")
    else:
        print(f"  Nenhum segmento de linha válido foi adicionado para '{nome_camada_debug}'.")

# Iterar sobre cada linha do DataFrame
for index, row in df.iterrows():
    print(f"\n--- Processando dados da Linha {index} do CSV ---")
    
    current_lat = row['Latitude']
    current_lon = row['Longitude']
    
    # Altitude do Ponto
    altitude_ponto_val = row[ALTITUDE_PONTO_COL]
    altitude_ponto_display = f"{altitude_ponto_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_ponto_val) else "N/A"
    
    # Informações da Linha 1 associada
    altitude_l1_val = row[ALTITUDE_LINHA1_COL]
    linha1_elev_display = f"{altitude_l1_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_l1_val) else "Nenhuma"

    # Informações da Linha 2 associada
    altitude_l2_val = row[ALTITUDE_LINHA2_COL]
    linha2_elev_display = f"{altitude_l2_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_l2_val) else "Nenhuma"

    # --- ALTERAÇÃO AQUI: Popup e Tooltip do Marcador do Ponto ---
    popup_html_marker = f"""
    <b>Ponto ID: {index}</b><br>
    ------------------------------------<br>
    <b>Coordenadas:</b><br>
    &nbsp;&nbsp;Latitude: {current_lat:.5f}<br>
    &nbsp;&nbsp;Longitude: {current_lon:.5f}<br>
    <b>Altitude do Ponto:</b> {altitude_ponto_display}<br>
    ------------------------------------<br>
    <b>Curvas de Nível Associadas:</b><br>
    &nbsp;&nbsp;<b>1ª Mais Próxima:</b> Elev. {linha1_elev_display}<br>
    &nbsp;&nbsp;<b>2ª Mais Próxima:</b> Elev. {linha2_elev_display}
    """
    # Opcional: Adicionar GIDs ou outros identificadores se úteis
    # if 'linha1_gid' in row and pd.notna(row['linha1_gid']):
    #     popup_html_marker += f"<br>&nbsp;&nbsp;(GID Linha 1: {row['linha1_gid']})"
    # if 'linha2_gid' in row and pd.notna(row['linha2_gid']):
    #     popup_html_marker += f"<br>&nbsp;&nbsp;(GID Linha 2: {row['linha2_gid']})"

    tooltip_text_marker = (f"Ponto {index} (Alt: {altitude_ponto_display}) | "
                           f"L1: {linha1_elev_display} | L2: {linha2_elev_display}")

    folium.Marker(
        location=[current_lat, current_lon],
        popup=folium.Popup(popup_html_marker, max_width=400), 
        tooltip=tooltip_text_marker,
        icon=folium.Icon(color='purple', icon='info-sign')
    ).add_to(fg_pontos_todos)
    # --- FIM DA ALTERAÇÃO ---

    print(f"  Ponto Principal (Linha CSV {index}): Lat={current_lat}, Lon={current_lon}, Alt Ponto={altitude_ponto_display}")
    print(f"    -> Associado à Linha 1 (Elev: {linha1_elev_display}), Linha 2 (Elev: {linha2_elev_display})")

    # Processar Curva de Nível 1 (esta parte permanece como antes)
    wkt_l1 = row['linha1_geometry']
    # altitude_l1_val já foi obtida acima para o popup do ponto, usamos a mesma aqui para a linha
    debug_name_l1 = f"Curva Nível 1 (CSV Linha {index})"
    adicionar_geometria_linha(wkt_l1, fg_curva1_todas, 'blue', debug_name_l1, altitude_l1_val)

    # Processar Curva de Nível 2 (esta parte permanece como antes)
    wkt_l2 = row['linha2_geometry']
    # altitude_l2_val já foi obtida acima para o popup do ponto, usamos a mesma aqui para a linha
    debug_name_l2 = f"Curva Nível 2 (CSV Linha {index})"
    adicionar_geometria_linha(wkt_l2, fg_curva2_todas, 'green', debug_name_l2, altitude_l2_val)

# Controle de Camadas
LayerControl().add_to(m)

# Salvamento do mapa
m.save(OUTPUT_MAP_FILE)
print(f"\nMapa salvo como '{OUTPUT_MAP_FILE}'. Abra este arquivo em um navegador para visualizar.")

Centro do mapa: Lat: -25.516848128860715, Lon: -49.23005693809807 (da primeira linha do CSV)

--- Processando dados da Linha 0 do CSV ---
  Ponto Principal (Linha CSV 0): Lat=-25.516848128860715, Lon=-49.23005693809807, Alt Ponto=887.9621448635928 m
    -> Associado à Linha 1 (Elev: 883.0 m), Linha 2 (Elev: 896.0 m)

Processando Curva Nível 1 (CSV Linha 0):
  WKT recebido: 'MULTILINESTRING ((675208.1430053711 7182001.752990723, 675207.1430053711 7182005.733032227, 675205.9...', Altitude associada à linha: 883.0 m
  Tipo de geometria parseada: MultiLineString
  1 segmento(s) de linha adicionado(s) para 'Curva Nível 1 (CSV Linha 0)'.
    Exemplo de primeiras coordenadas reprojetadas (Lat, Lon): (-25.468738598029258, -49.25721360195566)

Processando Curva Nível 2 (CSV Linha 0):
  WKT recebido: 'MULTILINESTRING ((674142.0330200195 7186799.692993164, 674144.5029907227 7186796.023010254, 674146.2...', Altitude associada à linha: 896.0 m
  Tipo de geometria parseada: MultiLineString
  1 segme

In [28]:
import folium
from folium import FeatureGroup, LayerControl
from shapely import wkt
# Não precisaremos mais de shapely.ops.nearest_points para esta abordagem
import pandas as pd
from pyproj import Transformer, CRS

# ==============================================================================
# CONFIGURAÇÕES DO USUÁRIO - AJUSTE CONFORME NECESSÁRIO
# ==============================================================================

SOURCE_EPSG = "EPSG:31982"
TARGET_EPSG = "EPSG:4326"

# Nomes das colunas de ATRIBUTOS no seu arquivo CSV
ALTITUDE_PONTO_COL = 'altimetria_interpolada'
ALTITUDE_LINHA1_COL = 'linha1_elevation'
ALTITUDE_LINHA2_COL = 'linha2_elevation'
ID_LINHA1_COL = 'linha1_gid' # <- NOVO: Coluna para o ID/GID da Linha 1
ID_LINHA2_COL = 'linha2_gid' # <- NOVO: Coluna para o ID/GID da Linha 2
UNIDADE_ALTITUDE = "m"

# Coluna com a geometria WKT das linhas (não do ponto original para esta versão)
# A coluna GEOMETRIA_PONTO_ORIGINAL_COL não é mais necessária para esta abordagem.

FILE_PATH_CSV = 'resultado1.csv'
OUTPUT_MAP_FILE = 'mapa_pontos_com_id_curvas.html'
# ==============================================================================

try:
    transformer = Transformer.from_crs(CRS.from_string(SOURCE_EPSG), CRS.from_string(TARGET_EPSG), always_xy=True)
except Exception as e:
    print(f"Erro ao criar o transformador de coordenadas: {e}")
    exit()
'''
try:
    df = pd.read_csv(FILE_PATH_CSV)
except Exception as e:
    print(f"Erro ao ler o CSV '{FILE_PATH_CSV}': {e}")
    exit()
'''
'''
ten = resultado_copia.head(50)

df = ten.copy()
'''

df = resultado_copia.copy()
if df.empty:
    print(f"DataFrame carregado de '{FILE_PATH_CSV}' está vazio.")
    exit()

# Colunas obrigatórias para esta funcionalidade
required_columns = ['Latitude', 'Longitude', # Coordenadas do Ponto
                    'linha1_geometry', 'linha2_geometry', # Geometrias das linhas associadas
                    ALTITUDE_PONTO_COL, ALTITUDE_LINHA1_COL, ALTITUDE_LINHA2_COL, # Altitudes
                    ID_LINHA1_COL, ID_LINHA2_COL] # IDs das linhas associadas
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    print(f"Erro: Colunas obrigatórias não encontradas: {', '.join(missing_cols)}")
    print(f"Colunas no CSV: {df.columns.tolist()}")
    print(f"Verifique os nomes das colunas definidos nas configurações.")
    exit()

map_center_latitude = df.loc[0, 'Latitude']
map_center_longitude = df.loc[0, 'Longitude']
print(f"Centro do mapa: Lat: {map_center_latitude}, Lon: {map_center_longitude}")

m = folium.Map(location=[map_center_latitude, map_center_longitude], zoom_start=12)

fg_pontos_todos = FeatureGroup(name='Pontos Principais', show=True)
m.add_child(fg_pontos_todos)
fg_curva1_todas = FeatureGroup(name='Curvas de Nível 1', show=True)
m.add_child(fg_curva1_todas)
fg_curva2_todas = FeatureGroup(name='Curvas de Nível 2', show=True)
m.add_child(fg_curva2_todas)
# Não precisamos mais dos FeatureGroups para linhas de conexão


def adicionar_geometria_linha(wkt_string, feature_group, cor_linha, nome_camada_debug, altitude_valor, id_linha_valor=None): # Adicionado id_linha_valor opcional
    print(f"\nProcessando {nome_camada_debug}:")
    if pd.isna(wkt_string) or not str(wkt_string).strip():
        print(f"  Aviso: String WKT para '{nome_camada_debug}' está vazia ou é NaN. Camada pulada.")
        return

    altitude_display = f"{altitude_valor} {UNIDADE_ALTITUDE}" if pd.notna(altitude_valor) else "N/A"
    id_display = f"ID: {id_linha_valor}" if pd.notna(id_linha_valor) else "ID: N/A" # Display para ID da linha

    popup_text_linha = f"Curva de Nível<br>{id_display}<br>Altitude: {altitude_display}"
    tooltip_text_linha = f"{id_display}, Alt: {altitude_display}"

    wkt_string_preview = str(wkt_string)[:100] + ('...' if len(str(wkt_string)) > 100 else '')
    print(f"  WKT recebido: '{wkt_string_preview}', {id_display}, Altitude da linha: {altitude_display}")

    try:
        geom = wkt.loads(str(wkt_string))
    except Exception as e:
        print(f"  Erro ao parsear WKT para '{nome_camada_debug}': {wkt_string_preview}. Erro: {e}")
        return

    if geom.is_empty:
        print(f"  Aviso: Geometria WKT parseada para '{nome_camada_debug}' está vazia. Camada pulada.")
        return
    print(f"  Tipo de geometria parseada: {geom.geom_type}")

    linhas_para_plotar_orig = []
    if geom.geom_type == 'LineString':
        linhas_para_plotar_orig.append(geom)
    elif geom.geom_type == 'MultiLineString':
        linhas_para_plotar_orig.extend(list(geom.geoms))
    else:
        print(f"  Aviso: Tipo de geometria não suportado '{geom.geom_type}' para '{nome_camada_debug}'. Camada pulada.")
        return

    if not linhas_para_plotar_orig:
        print(f"  Aviso: Nenhuma linha individual encontrada em '{nome_camada_debug}'.")
        return
        
    segmentos_adicionados = 0
    coords_reprojetadas_folium_exemplo = []
    for i, linha_orig in enumerate(linhas_para_plotar_orig):
        if linha_orig.is_empty:
            print(f"  Aviso: Segmento {i} (original) de '{nome_camada_debug}' está vazio.")
            continue
        
        coords_originais_projetadas = list(linha_orig.coords)
        if not coords_originais_projetadas:
            print(f"  Aviso: Segmento {i} (original) de '{nome_camada_debug}' não possui coordenadas.")
            continue
        
        coords_reprojetadas_folium = []
        try:
            for x_proj, y_proj in coords_originais_projetadas:
                lon_deg, lat_deg = transformer.transform(x_proj, y_proj)
                coords_reprojetadas_folium.append((lat_deg, lon_deg))
        except Exception as e:
            print(f"    Erro durante reprojeção do segmento {i} de '{nome_camada_debug}': {e}")
            continue

        if not coords_reprojetadas_folium:
            print(f"  Aviso: Segmento {i} de '{nome_camada_debug}' não possui coordenadas após reprojeção.")
            continue
        
        if not coords_reprojetadas_folium_exemplo:
            coords_reprojetadas_folium_exemplo = coords_reprojetadas_folium

        folium.PolyLine(
            coords_reprojetadas_folium, color=cor_linha, weight=2.5, opacity=0.8,
            popup=folium.Popup(popup_text_linha, max_width=250), # Usando folium.Popup
            tooltip=tooltip_text_linha
        ).add_to(feature_group)
        segmentos_adicionados +=1
    
    if segmentos_adicionados > 0:
        print(f"  {segmentos_adicionados} segmento(s) de linha adicionado(s) para '{nome_camada_debug}'.")
        if coords_reprojetadas_folium_exemplo and coords_reprojetadas_folium_exemplo[0]:
             print(f"    Exemplo de primeiras coordenadas reprojetadas (Lat, Lon): {coords_reprojetadas_folium_exemplo[0]}")
    else:
        print(f"  Nenhum segmento de linha válido foi adicionado para '{nome_camada_debug}'.")


for index, row in df.iterrows():
    print(f"\n--- Processando dados da Linha {index} do CSV ---")
    
    current_lat = row['Latitude']
    current_lon = row['Longitude']
    
    altitude_ponto_val = row[ALTITUDE_PONTO_COL]
    altitude_ponto_display = f"{altitude_ponto_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_ponto_val) else "N/A"
    
    # Informações da Linha 1 associada
    altitude_l1_val = row[ALTITUDE_LINHA1_COL]
    id_l1_val = row[ID_LINHA1_COL]
    linha1_elev_display = f"{altitude_l1_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_l1_val) else "N/A"
    linha1_id_display = str(id_l1_val) if pd.notna(id_l1_val) else "N/A"

    # Informações da Linha 2 associada
    altitude_l2_val = row[ALTITUDE_LINHA2_COL]
    id_l2_val = row[ID_LINHA2_COL]
    linha2_elev_display = f"{altitude_l2_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_l2_val) else "N/A"
    linha2_id_display = str(id_l2_val) if pd.notna(id_l2_val) else "N/A"

    # --- Popup e Tooltip do Marcador do Ponto com IDs das Curvas ---
    popup_html_marker = f"""
    <b>Ponto ID {index}</b> (Lat: {current_lat:.5f}, Lon: {current_lon:.5f})<br>
    Altitude do Ponto: {altitude_ponto_display}<br>
    <hr style="margin: 5px 0;">
    <b>Curvas de Nível Associadas:</b><br>
    <u>1ª Mais Próxima:</u><br>
    &nbsp;&nbsp;ID da Curva: {linha1_id_display}<br>
    &nbsp;&nbsp;Elevação: {linha1_elev_display}<br>
    <u>2ª Mais Próxima:</u><br>
    &nbsp;&nbsp;ID da Curva: {linha2_id_display}<br>
    &nbsp;&nbsp;Elevação: {linha2_elev_display}
    """

    tooltip_text_marker = (f"Ponto {index} (Alt: {altitude_ponto_display}) | "
                           f"L1 ID: {linha1_id_display} (Elev: {linha1_elev_display}) | "
                           f"L2 ID: {linha2_id_display} (Elev: {linha2_elev_display})")

    folium.Marker(
        location=[current_lat, current_lon],
        popup=folium.Popup(popup_html_marker, max_width=400), 
        tooltip=tooltip_text_marker,
        icon=folium.Icon(color='purple', icon='info-sign')
    ).add_to(fg_pontos_todos)
    # --- FIM DA ALTERAÇÃO Popup e Tooltip ---

    print(f"  Ponto Principal (Linha CSV {index}): Lat={current_lat}, Lon={current_lon}, Alt Ponto={altitude_ponto_display}")
    print(f"    -> Associado à Curva 1 (ID: {linha1_id_display}, Elev: {linha1_elev_display})")
    print(f"    -> Associado à Curva 2 (ID: {linha2_id_display}, Elev: {linha2_elev_display})")

    # Adiciona as geometrias das curvas de nível ao mapa
    # Passando também o ID da linha para a função, para que possa ser incluído no popup da própria linha
    adicionar_geometria_linha(row['linha1_geometry'], fg_curva1_todas, 'blue', 
                              f"Curva Nível 1 (CSV Linha {index})", altitude_l1_val, id_l1_val)
    adicionar_geometria_linha(row['linha2_geometry'], fg_curva2_todas, 'green', 
                              f"Curva Nível 2 (CSV Linha {index})", altitude_l2_val, id_l2_val)

    # A lógica para desenhar linhas de conexão foi removida.

LayerControl().add_to(m)
m.save(OUTPUT_MAP_FILE)
print(f"\nMapa salvo como '{OUTPUT_MAP_FILE}'.")

Centro do mapa: Lat: -25.40663119894788, Lon: -49.25216384754277

--- Processando dados da Linha 0 do CSV ---
  Ponto Principal (Linha CSV 0): Lat=-25.40663119894788, Lon=-49.25216384754277, Alt Ponto=912.3155675899501 m
    -> Associado à Curva 1 (ID: 122759, Elev: 912.0 m)
    -> Associado à Curva 2 (ID: 122761, Elev: 913.0 m)

Processando Curva Nível 1 (CSV Linha 0):
  WKT recebido: 'MULTILINESTRING ((675768.6744384766 7188999.692993164, 675765.6829833984 7188997.37298584, 675766.27...', ID: 122759, Altitude da linha: 912.0 m
  Tipo de geometria parseada: MultiLineString
  1 segmento(s) de linha adicionado(s) para 'Curva Nível 1 (CSV Linha 0)'.
    Exemplo de primeiras coordenadas reprojetadas (Lat, Lon): (-25.405506523340453, -49.25255188210982)

Processando Curva Nível 2 (CSV Linha 0):
  WKT recebido: 'MULTILINESTRING ((675637.999206543 7188999.692993164, 675637.9930419922 7188999.562988281, 675634.66...', ID: 122761, Altitude da linha: 913.0 m
  Tipo de geometria parseada: MultiL

In [10]:
import folium
from folium import FeatureGroup, LayerControl
from folium.plugins import HeatMap
import branca.colormap as cm
from shapely import wkt
import pandas as pd
from pyproj import Transformer, CRS

# ==============================================================================
# CONFIGURAÇÕES DO USUÁRIO - AJUSTE CONFORME NECESSÁRIO
# ==============================================================================

SOURCE_EPSG = "EPSG:31982"
TARGET_EPSG = "EPSG:4326"

# Nomes das colunas de ATRIBUTOS no seu arquivo CSV
ALTITUDE_PONTO_COL = 'altimetria_interpolada'
ALTITUDE_LINHA1_COL = 'linha1_elevation'
ALTITUDE_LINHA2_COL = 'linha2_elevation'
ID_LINHA1_COL = 'linha1_gid'
ID_LINHA2_COL = 'linha2_gid'
UNIDADE_ALTITUDE = "m"

# Configurações para destaque de variação altimétrica
TOP_N_PONTOS = 50  # Número de pontos com maior variação a destacar
COR_BAIXA_VARIACAO = 'green'
COR_ALTA_VARIACAO = 'red'

FILE_PATH_CSV = 'resultado1.csv'
OUTPUT_MAP_FILE = 'mapa_variacao_altimetrica.html'
# ==============================================================================

# Criar transformador de coordenadas
try:
    transformer = Transformer.from_crs(CRS.from_string(SOURCE_EPSG), CRS.from_string(TARGET_EPSG), always_xy=True)
except Exception as e:
    print(f"Erro ao criar o transformador de coordenadas: {e}")
    exit()

df = resultado_copia.copy()

if df.empty:
    print(f"DataFrame carregado de '{FILE_PATH_CSV}' está vazio.")
    exit()

# Verificar colunas obrigatórias
required_columns = ['Latitude', 'Longitude', 
                    'linha1_geometry', 'linha2_geometry',
                    ALTITUDE_PONTO_COL, ALTITUDE_LINHA1_COL, ALTITUDE_LINHA2_COL,
                    ID_LINHA1_COL, ID_LINHA2_COL]

missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    print(f"Erro: Colunas obrigatórias não encontradas: {', '.join(missing_cols)}")
    print(f"Colunas no CSV: {df.columns.tolist()}")
    exit()

# Calcular variação altimétrica
df['variacao_altimetrica'] = abs(df[ALTITUDE_LINHA1_COL] - df[ALTITUDE_LINHA2_COL])

# Identificar pontos com maior variação
df_ordenado = df.sort_values('variacao_altimetrica', ascending=False)
top_variacoes = df_ordenado.head(TOP_N_PONTOS)

# Criar mapa
map_center_latitude = df['Latitude'].mean()
map_center_longitude = df['Longitude'].mean()
print(f"Centro do mapa: Lat: {map_center_latitude}, Lon: {map_center_longitude}")

m = folium.Map(location=[map_center_latitude, map_center_longitude], 
               zoom_start=12, 
               tiles='OpenStreetMap')

# ==============================================================================
# FUNÇÃO PARA ADICIONAR GEOMETRIAS DE LINHA
# ==============================================================================
def adicionar_geometria_linha(wkt_string, feature_group, cor_linha, nome_camada_debug, altitude_valor, id_linha_valor=None):
    if pd.isna(wkt_string) or not str(wkt_string).strip():
        return

    altitude_display = f"{altitude_valor} {UNIDADE_ALTITUDE}" if pd.notna(altitude_valor) else "N/A"
    id_display = f"ID: {id_linha_valor}" if pd.notna(id_linha_valor) else "ID: N/A"

    popup_text_linha = f"Curva de Nível<br>{id_display}<br>Altitude: {altitude_display}"
    tooltip_text_linha = f"{id_display}, Alt: {altitude_display}"

    try:
        geom = wkt.loads(str(wkt_string))
    except Exception as e:
        return

    if geom.is_empty:
        return

    linhas_para_plotar_orig = []
    if geom.geom_type == 'LineString':
        linhas_para_plotar_orig.append(geom)
    elif geom.geom_type == 'MultiLineString':
        linhas_para_plotar_orig.extend(list(geom.geoms))
    else:
        return
        
    for linha_orig in linhas_para_plotar_orig:
        if linha_orig.is_empty:
            continue
        
        coords_originais_projetadas = list(linha_orig.coords)
        if not coords_originais_projetadas:
            continue
        
        coords_reprojetadas_folium = []
        try:
            for x_proj, y_proj in coords_originais_projetadas:
                lon_deg, lat_deg = transformer.transform(x_proj, y_proj)
                coords_reprojetadas_folium.append((lat_deg, lon_deg))
        except Exception as e:
            continue

        if not coords_reprojetadas_folium:
            continue
        
        folium.PolyLine(
            coords_reprojetadas_folium, color=cor_linha, weight=2.5, opacity=0.8,
            popup=folium.Popup(popup_text_linha, max_width=250),
            tooltip=tooltip_text_linha
        ).add_to(feature_group)

# ==============================================================================
# CONFIGURAR CAMADAS DO MAPA
# ==============================================================================

# Camada para todos os pontos
fg_pontos_todos = FeatureGroup(name='Todos os Pontos', show=True)
m.add_child(fg_pontos_todos)

# Camada para curvas de nível
fg_curva1_todas = FeatureGroup(name='Curvas de Nível 1', show=False)
m.add_child(fg_curva1_todas)
fg_curva2_todas = FeatureGroup(name='Curvas de Nível 2', show=False)
m.add_child(fg_curva2_todas)

# Camada para pontos de alta variação
fg_variacoes = FeatureGroup(name=f'Top {TOP_N_PONTOS} Variações', show=True)
m.add_child(fg_variacoes)

# Camada para heatmap
fg_heatmap = FeatureGroup(name='Heatmap de Variação', show=True)
m.add_child(fg_heatmap)

# ==============================================================================
# ADICIONAR DADOS AO MAPA
# ==============================================================================

# Criar colormap para as variações
if df['variacao_altimetrica'].notna().any():
    min_variacao = df['variacao_altimetrica'].min()
    max_variacao = df['variacao_altimetrica'].max()
    colormap = cm.LinearColormap(
        [COR_BAIXA_VARIACAO, COR_ALTA_VARIACAO],
        vmin=min_variacao,
        vmax=max_variacao
    )
    colormap.caption = 'Variação Altimétrica (m)'
    m.add_child(colormap)

# Adicionar heatmap
heat_data = [[row['Latitude'], row['Longitude'], row['variacao_altimetrica']] 
             for _, row in df.iterrows() if pd.notna(row['variacao_altimetrica'])]

if heat_data:
    HeatMap(
        heat_data,
        name='Variação Altimétrica',
        min_opacity=0.3,
        max_val=df['variacao_altimetrica'].quantile(0.95),
        radius=15,
        blur=15,
        gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}
    ).add_to(fg_heatmap)

# Processar todos os pontos
for index, row in df.iterrows():
    current_lat = row['Latitude']
    current_lon = row['Longitude']
    
    altitude_ponto_val = row[ALTITUDE_PONTO_COL]
    altitude_ponto_display = f"{altitude_ponto_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_ponto_val) else "N/A"
    
    # Informações das curvas associadas
    altitude_l1_val = row[ALTITUDE_LINHA1_COL]
    id_l1_val = row[ID_LINHA1_COL]
    linha1_elev_display = f"{altitude_l1_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_l1_val) else "N/A"
    linha1_id_display = str(id_l1_val) if pd.notna(id_l1_val) else "N/A"

    altitude_l2_val = row[ALTITUDE_LINHA2_COL]
    id_l2_val = row[ID_LINHA2_COL]
    linha2_elev_display = f"{altitude_l2_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_l2_val) else "N/A"
    linha2_id_display = str(id_l2_val) if pd.notna(id_l2_val) else "N/A"
    
    variacao_val = row['variacao_altimetrica'] if 'variacao_altimetrica' in row else 0
    variacao_display = f"{variacao_val:.1f} {UNIDADE_ALTITUDE}" if pd.notna(variacao_val) else "N/A"

    # Popup para todos os pontos
    popup_html_marker = f"""
    <b>Ponto {index}</b><br>
    Variação Altimétrica: <b>{variacao_display}</b><br>
    Altitude do Ponto: {altitude_ponto_display}<br>
    <hr style="margin: 5px 0;">
    <b>Curvas de Nível Associadas:</b><br>
    <u>1ª Mais Próxima:</u><br>
    &nbsp;&nbsp;ID: {linha1_id_display}<br>
    &nbsp;&nbsp;Elevação: {linha1_elev_display}<br>
    <u>2ª Mais Próxima:</u><br>
    &nbsp;&nbsp;ID: {linha2_id_display}<br>
    &nbsp;&nbsp;Elevação: {linha2_elev_display}
    """

    # Adicionar marcador para todos os pontos
    folium.CircleMarker(
        location=[current_lat, current_lon],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        popup=folium.Popup(popup_html_marker, max_width=400),
        tooltip=f"Ponto {index} (Variação: {variacao_display})"
    ).add_to(fg_pontos_todos)
    
    # Adicionar geometrias das curvas de nível
    adicionar_geometria_linha(row['linha1_geometry'], fg_curva1_todas, 'blue', 
                              f"Curva Nível 1", altitude_l1_val, id_l1_val)
    adicionar_geometria_linha(row['linha2_geometry'], fg_curva2_todas, 'green', 
                              f"Curva Nível 2", altitude_l2_val, id_l2_val)

# Adicionar pontos de alta variação
for index, row in top_variacoes.iterrows():
    current_lat = row['Latitude']
    current_lon = row['Longitude']
    variacao = row['variacao_altimetrica']
    
    # Popup detalhado para alta variação
    popup_html = f"""
    <b>Ponto {index} - Alta Variação</b><br>
    Variação Altimétrica: <b>{variacao:.1f} m</b><br>
    <hr>
    <b>Curva 1:</b> {row[ALTITUDE_LINHA1_COL]} m (ID: {row[ID_LINHA1_COL]})<br>
    <b>Curva 2:</b> {row[ALTITUDE_LINHA2_COL]} m (ID: {row[ID_LINHA2_COL]})<br>
    <b>Diferença:</b> {abs(row[ALTITUDE_LINHA1_COL] - row[ALTITUDE_LINHA2_COL]):.1f} m
    """
    
    # Adicionar marcador especial
    folium.CircleMarker(
        location=[current_lat, current_lon],
        radius=8 + (variacao / 5),  # Tamanho proporcional à variação
        color=colormap(variacao),
        fill=True,
        fill_color=colormap(variacao),
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"Variação: {variacao:.1f} m"
    ).add_to(fg_variacoes)

# ==============================================================================
# FINALIZAR E SALVAR O MAPA
# ==============================================================================

# Adicionar controle de camadas
LayerControl(collapsed=False).add_to(m)

# Adicionar título
title_html = '''
     <h3 align="center" style="font-size:16px"><b>Mapa de Variação Altimétrica</b></h3>
     <p align="center">Pontos vermelhos: maiores variações entre curvas de nível</p>
     '''
m.get_root().html.add_child(folium.Element(title_html))

# Salvar mapa
m.save(OUTPUT_MAP_FILE)
print(f"\nMapa salvo como '{OUTPUT_MAP_FILE}'.")
print(f"Total de pontos processados: {len(df)}")
print(f"Pontos com maior variação destacados: {TOP_N_PONTOS}")
print(f"Máxima variação encontrada: {df['variacao_altimetrica'].max():.1f} m")

Centro do mapa: Lat: -25.472152698921207, Lon: -49.25169232440742


/tmp/ipykernel_7858/298245317.py:171: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(



Mapa salvo como 'mapa_variacao_altimetrica.html'.
Total de pontos processados: 560
Pontos com maior variação destacados: 50
Máxima variação encontrada: 2.0 m


In [12]:
import folium
from folium import FeatureGroup, LayerControl
# HeatMap não é mais usado, então a importação foi removida
import branca.colormap as cm
from shapely import wkt
import pandas as pd
from pyproj import Transformer, CRS

# ==============================================================================
# CONFIGURAÇÕES DO USUÁRIO - AJUSTE CONFORME NECESSÁRIO
# ==============================================================================

SOURCE_EPSG = "EPSG:31982"
TARGET_EPSG = "EPSG:4326"

# Nomes das colunas de ATRIBUTOS no seu arquivo CSV
ALTITUDE_PONTO_COL = 'altimetria_interpolada'
ALTITUDE_LINHA1_COL = 'linha1_elevation'
ALTITUDE_LINHA2_COL = 'linha2_elevation'
ID_LINHA1_COL = 'linha1_gid'
ID_LINHA2_COL = 'linha2_gid'
UNIDADE_ALTITUDE = "m"

# Configurações para destaque de variação altimétrica
COR_BAIXA_VARIACAO = 'green'
COR_ALTA_VARIACAO = 'red'

# ATENÇÃO: Assumindo que seu DataFrame já está carregado na variável 'resultado_copia'
# Se estiver lendo de um CSV, descomente a linha abaixo e ajuste o nome do arquivo
# FILE_PATH_CSV = 'resultado1.csv'
OUTPUT_MAP_FILE = 'mapa_variacao_altimetrica.html'
# ==============================================================================

# Criar transformador de coordenadas
try:
    transformer = Transformer.from_crs(CRS.from_string(SOURCE_EPSG), CRS.from_string(TARGET_EPSG), always_xy=True)
except Exception as e:
    print(f"Erro ao criar o transformador de coordenadas: {e}")
    exit()

# Copia o DataFrame da variável existente no seu ambiente
# Se for ler de um arquivo, use: df = pd.read_csv(FILE_PATH_CSV)
df = resultado_copia.copy()

if df.empty:
    print("O DataFrame está vazio.")
    exit()

# Verificar colunas obrigatórias
required_columns = ['Latitude', 'Longitude',
                    'linha1_geometry', 'linha2_geometry',
                    ALTITUDE_PONTO_COL, ALTITUDE_LINHA1_COL, ALTITUDE_LINHA2_COL,
                    ID_LINHA1_COL, ID_LINHA2_COL]

missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    print(f"Erro: Colunas obrigatórias não encontradas: {', '.join(missing_cols)}")
    print(f"Colunas existentes: {df.columns.tolist()}")
    exit()

# Calcular variação altimétrica
df['variacao_altimetrica'] = abs(df[ALTITUDE_LINHA1_COL] - df[ALTITUDE_LINHA2_COL])

# Criar mapa
map_center_latitude = df['Latitude'].mean()
map_center_longitude = df['Longitude'].mean()
print(f"Centro do mapa: Lat: {map_center_latitude}, Lon: {map_center_longitude}")

m = folium.Map(location=[map_center_latitude, map_center_longitude],
               zoom_start=12,
               tiles='OpenStreetMap')

# ==============================================================================
# FUNÇÃO PARA ADICIONAR GEOMETRIAS DE LINHA (sem alterações)
# ==============================================================================
def adicionar_geometria_linha(wkt_string, feature_group, cor_linha, nome_camada_debug, altitude_valor, id_linha_valor=None):
    if pd.isna(wkt_string) or not str(wkt_string).strip():
        return

    altitude_display = f"{altitude_valor} {UNIDADE_ALTITUDE}" if pd.notna(altitude_valor) else "N/A"
    id_display = f"ID: {id_linha_valor}" if pd.notna(id_linha_valor) else "ID: N/A"

    popup_text_linha = f"Curva de Nível<br>{id_display}<br>Altitude: {altitude_display}"
    tooltip_text_linha = f"{id_display}, Alt: {altitude_display}"

    try:
        geom = wkt.loads(str(wkt_string))
    except Exception as e:
        return

    if geom.is_empty:
        return

    linhas_para_plotar_orig = []
    if geom.geom_type == 'LineString':
        linhas_para_plotar_orig.append(geom)
    elif geom.geom_type == 'MultiLineString':
        linhas_para_plotar_orig.extend(list(geom.geoms))
    else:
        return

    for linha_orig in linhas_para_plotar_orig:
        if linha_orig.is_empty:
            continue

        coords_originais_projetadas = list(linha_orig.coords)
        if not coords_originais_projetadas:
            continue

        coords_reprojetadas_folium = []
        try:
            for x_proj, y_proj in coords_originais_projetadas:
                lon_deg, lat_deg = transformer.transform(x_proj, y_proj)
                coords_reprojetadas_folium.append((lat_deg, lon_deg))
        except Exception as e:
            continue

        if not coords_reprojetadas_folium:
            continue

        folium.PolyLine(
            coords_reprojetadas_folium, color=cor_linha, weight=2.5, opacity=0.8,
            popup=folium.Popup(popup_text_linha, max_width=250),
            tooltip=tooltip_text_linha
        ).add_to(feature_group)

# ==============================================================================
# CONFIGURAR CAMADAS DO MAPA
# ==============================================================================

# Camada para todos os pontos (serão todos coloridos)
fg_pontos_todos = FeatureGroup(name='Todos os Pontos por Variação', show=True)
m.add_child(fg_pontos_todos)

# Camada para curvas de nível
fg_curva1_todas = FeatureGroup(name='Curvas de Nível 1', show=False)
m.add_child(fg_curva1_todas)
fg_curva2_todas = FeatureGroup(name='Curvas de Nível 2', show=False)
m.add_child(fg_curva2_todas)

# A CAMADA DO HEATMAP FOI REMOVIDA DAQUI

# ==============================================================================
# ADICIONAR DADOS AO MAPA
# ==============================================================================

# Criar colormap para as variações de verde a vermelho
if df['variacao_altimetrica'].notna().any():
    min_variacao = df['variacao_altimetrica'].min()
    max_variacao = df['variacao_altimetrica'].max()
    colormap = cm.LinearColormap(
        [COR_BAIXA_VARIACAO, COR_ALTA_VARIACAO],
        vmin=min_variacao,
        vmax=max_variacao
    )
    colormap.caption = 'Variação Altimétrica (m)'
    m.add_child(colormap)

# O BLOCO DE CÓDIGO DO HEATMAP FOI REMOVIDO DAQUI

# Processar e adicionar TODOS os pontos ao mapa, já coloridos
for index, row in df.iterrows():
    current_lat = row['Latitude']
    current_lon = row['Longitude']

    altitude_ponto_val = row[ALTITUDE_PONTO_COL]
    altitude_ponto_display = f"{altitude_ponto_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_ponto_val) else "N/A"

    altitude_l1_val = row[ALTITUDE_LINHA1_COL]
    id_l1_val = row[ID_LINHA1_COL]
    linha1_elev_display = f"{altitude_l1_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_l1_val) else "N/A"
    linha1_id_display = str(id_l1_val) if pd.notna(id_l1_val) else "N/A"

    altitude_l2_val = row[ALTITUDE_LINHA2_COL]
    id_l2_val = row[ID_LINHA2_COL]
    linha2_elev_display = f"{altitude_l2_val} {UNIDADE_ALTITUDE}" if pd.notna(altitude_l2_val) else "N/A"
    linha2_id_display = str(id_l2_val) if pd.notna(id_l2_val) else "N/A"

    variacao_val = row['variacao_altimetrica']
    variacao_display = f"{variacao_val:.1f} {UNIDADE_ALTITUDE}" if pd.notna(variacao_val) else "N/A"

    # Popup para todos os pontos
    popup_html_marker = f"""
    <b>Ponto {index}</b><br>
    Variação Altimétrica: <b>{variacao_display}</b><br>
    Altitude do Ponto: {altitude_ponto_display}<br>
    <hr style="margin: 5px 0;">
    <b>Curvas de Nível Associadas:</b><br>
    <u>1ª Mais Próxima:</u><br>
    &nbsp;&nbsp;ID: {linha1_id_display}<br>
    &nbsp;&nbsp;Elevação: {linha1_elev_display}<br>
    <u>2ª Mais Próxima:</u><br>
    &nbsp;&nbsp;ID: {linha2_id_display}<br>
    &nbsp;&nbsp;Elevação: {linha2_elev_display}
    """

    # Adicionar marcador para todos os pontos, usando a cor da variação
    folium.CircleMarker(
        location=[current_lat, current_lon],
        radius=5,
        color=colormap(variacao_val),
        fill=True,
        fill_color=colormap(variacao_val),
        fill_opacity=0.7,
        popup=folium.Popup(popup_html_marker, max_width=400),
        tooltip=f"Ponto {index} (Variação: {variacao_display})"
    ).add_to(fg_pontos_todos)

    # Adicionar geometrias das curvas de nível
    adicionar_geometria_linha(row['linha1_geometry'], fg_curva1_todas, 'blue',
                              f"Curva Nível 1", altitude_l1_val, id_l1_val)
    adicionar_geometria_linha(row['linha2_geometry'], fg_curva2_todas, 'green',
                              f"Curva Nível 2", altitude_l2_val, id_l2_val)

# ==============================================================================
# FINALIZAR E SALVAR O MAPA
# ==============================================================================

# Adicionar controle de camadas
LayerControl(collapsed=False).add_to(m)

# Adicionar título
title_html = '''
     <h3 align="center" style="font-size:16px"><b>Mapa de Variação Altimétrica</b></h3>
     <p align="center">Cada ponto é colorido pela sua variação (Verde: Baixa, Vermelho: Alta)</p>
     '''
m.get_root().html.add_child(folium.Element(title_html))

# Salvar mapa
m.save(OUTPUT_MAP_FILE)
print(f"\nMapa salvo como '{OUTPUT_MAP_FILE}'.")
print(f"Total de pontos processados: {len(df)}")
print(f"Máxima variação encontrada: {df['variacao_altimetrica'].max():.1f} m")

Centro do mapa: Lat: -25.472152698921207, Lon: -49.25169232440742

Mapa salvo como 'mapa_variacao_altimetrica.html'.
Total de pontos processados: 560
Máxima variação encontrada: 2.0 m
